In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
scores_phase = pd.read_csv('../data/ec109_mismatch_old_match_old_phase_scores.csv')

In [2]:
df = pd.read_csv('../data/mismatch_old_behav_ec109.csv')

In [6]:
def normalize_data(df, keep_pow, keep_phase):
    # z-score within block
    gb = df.groupby(['elecs', 'freqs', 'blockno'])
    if keep_pow:
        df['zpow'] = gb['pow'].apply(zscore)
    if keep_phase:
        df['sin'] = gb['phase'].apply(np.sin)
        df['cos'] = gb['phase'].apply(np.cos)

    df.drop(['pow', 'phase'], inplace=True, axis=1)
    return df

def zscore(x):
    ''' computes z score on value within a series of data '''
    z = (x - np.mean(x)) / np.std(x)
    return z
def downsample_timeseries(timearray, newsamplerate):
    alltimes = timearray.time.unique()
    times_to_keep = alltimes[0:len(alltimes):newsamplerate]
    downsampled = timearray.set_index('time')
    downsampled = downsampled.loc[times_to_keep,:]
    downsampled = downsampled.reset_index()
    return downsampled

In [ ]:
df = normalize_data(df, keep_pow=True, keep_phase=True)

In [8]:
df = downsample_timeseries(df, 10)

In [12]:
df

,time,events,blockno,elecs,freqs,response,view1,view2,view3,zpow,sin,cos
0,1,1,1,1,1,2,0.509735,0.300050,0.035447,-1.178307,-0.891686,-0.452654
1,1,2,1,1,1,2,0.575637,0.300050,0.035447,-1.889750,0.703466,0.710729
2,1,3,1,1,1,2,0.583125,0.351473,0.035447,-0.716471,-0.851743,-0.523960
3,1,4,1,1,1,2,0.583125,0.303545,0.104843,-0.240946,-0.157656,0.987494
4,1,5,1,1,1,2,0.583125,0.250624,0.137294,-1.246163,-0.984338,-0.176294
5,1,6,1,1,1,2,0.583125,0.100849,0.137294,-0.590211,-0.266463,-0.963845
6,1,7,1,1,1,2,0.583125,0.100849,0.137294,-0.062463,-0.128450,0.991716
7,1,8,1,1,1,2,0.108337,0.099850,0.126810,-0.212878,0.985507,0.169635
8,1,9,1,1,1,1,0.222167,0.416375,0.156266,-0.537026,-0.383824,0.923406
9,1,10,1,1,1,1,0.222167,0.461308,0.156266,-0.066280,0.841594,-0.540110


In [99]:
df['view1-3'] = df['view1'] - df['view3']

In [113]:
y= df.loc[:,'view1-3']
X= df.loc[:,['elecs', 'blockno']]

In [114]:
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=.2)

In [84]:
sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'brier_score_loss',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'mutual_info_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'v_measure_score']

In [116]:
alphas = np.array([10,50,100])
ridgecv = RidgeCV(alphas=alphas, fit_intercept=False, cv=None, scoring ='neg_mean_squared_error')
ridgecv.fit(x_train, y_train)

RidgeCV(alphas=array([ 10,  50, 100]), cv=None, fit_intercept=False,
    gcv_mode=None, normalize=False, scoring='neg_mean_squared_error',
    store_cv_values=False)

In [117]:
ridgecv.intercept_

0.0

In [118]:
y_preds = ridgecv.predict(x_test)
y_preds

array([2.37508502e-07, 2.37508502e-07, 2.37508502e-07, ...,
       2.37508502e-07, 2.37508502e-07, 2.37508502e-07])

In [119]:
mean_squared_error(y_test, y_preds)

0.0887066931940309

In [105]:
dic = {'preds':y_preds, 'labels':y_test}
pd.DataFrame(dic)

,labels,preds
8174676,-0.072891,0.146806
14776699,0.302047,0.155566
5372959,-0.018472,0.152711
21242853,-0.123814,0.145315
10502008,0.692461,0.147704
21286489,-0.018472,0.154038
20328667,0.259611,0.149462
12237454,0.445831,0.157441
10725427,0.030954,0.157873
5370496,-0.165751,0.156818


In [60]:
newdf = pd.DataFrame()
for inds, df in data.groupby(by=['events', 'elecs', 'freqs', 'blockno']):
    tmp = df.iloc[0:751,:]
    break
    tmp = tmp.resample('25ns', on='datetime').mean()
    tmp.reset_index(drop=True, inplace=True)
    newdf = pd.concat([newdf, tmp])
newdf = newdf.round({'time':0})    
newdf.reset_index(drop=True, inplace=True)    

In [ ]:
dsrate=15
ds_string = f'{dsrate}ns'


In [33]:
x = pd.DataFrame(np.arange(-750,751,step=2),index=np.arange(1,752), columns=['time'])
x

,time
1,-750
2,-748
3,-746
4,-744
5,-742
6,-740
7,-738
8,-736
9,-734
10,-732


In [48]:
xx = tmp.resample('15ns', on='datetime').mean()
xx = xx.round({'time':0})

In [50]:
t_idx = np.arange(1,126)
t_idx

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125])

In [54]:
xx.loc[(xx['time'].isin(t_idx))]

,events,blockno,elecs,freqs,time,pow,phase,response,view1,view2,view3
datetime,,,,,,,,,,,
1970-01-01 00:00:00.000000000,1.0,1.0,1.0,1.0,8.0,2.389981,-1.849062,2.0,0.509735,0.30005,0.035447
1970-01-01 00:00:00.000000015,1.0,1.0,1.0,1.0,22.0,2.217858,-1.447794,2.0,0.509735,0.30005,0.035447
1970-01-01 00:00:00.000000030,1.0,1.0,1.0,1.0,37.0,2.042263,-1.078791,2.0,0.509735,0.30005,0.035447
1970-01-01 00:00:00.000000045,1.0,1.0,1.0,1.0,52.0,1.884469,-0.760976,2.0,0.509735,0.30005,0.035447
1970-01-01 00:00:00.000000060,1.0,1.0,1.0,1.0,67.0,1.766920,-0.489638,2.0,0.509735,0.30005,0.035447
1970-01-01 00:00:00.000000075,1.0,1.0,1.0,1.0,82.0,1.710398,-0.239011,2.0,0.509735,0.30005,0.035447
1970-01-01 00:00:00.000000090,1.0,1.0,1.0,1.0,97.0,1.719279,0.015471,2.0,0.509735,0.30005,0.035447
1970-01-01 00:00:00.000000105,1.0,1.0,1.0,1.0,112.0,1.786889,0.306289,2.0,0.509735,0.30005,0.035447


In [ ]:
sns.barplot(x='starttime', y='auc_z', hue='startfreq', data=scores)

In [64]:
df

,events,blockno,elecs,freqs,time,pow,phase,response,view1,view2,view3,datetime
0,1,1,1,1,1,2.464962,-2.040536,2,0.509735,0.30005,0.035447,1970-01-01 00:00:00.000000001
35100,1,1,1,1,2,2.453640,-2.010732,2,0.509735,0.30005,0.035447,1970-01-01 00:00:00.000000002
70200,1,1,1,1,3,2.442230,-1.980846,2,0.509735,0.30005,0.035447,1970-01-01 00:00:00.000000003
105300,1,1,1,1,4,2.430764,-1.950999,2,0.509735,0.30005,0.035447,1970-01-01 00:00:00.000000004
140400,1,1,1,1,5,2.419358,-1.921266,2,0.509735,0.30005,0.035447,1970-01-01 00:00:00.000000005
175500,1,1,1,1,6,2.407965,-1.891733,2,0.509735,0.30005,0.035447,1970-01-01 00:00:00.000000006
210600,1,1,1,1,7,2.396432,-1.862377,2,0.509735,0.30005,0.035447,1970-01-01 00:00:00.000000007
245700,1,1,1,1,8,2.384698,-1.833303,2,0.509735,0.30005,0.035447,1970-01-01 00:00:00.000000008
280800,1,1,1,1,9,2.372925,-1.804287,2,0.509735,0.30005,0.035447,1970-01-01 00:00:00.000000009
315900,1,1,1,1,10,2.361011,-1.775453,2,0.509735,0.30005,0.035447,1970-01-01 00:00:00.000000010


In [ ]:
scores.auc_z.dtype

In [ ]:
r1 = data[data['response']==1]

In [ ]:
r1['class']=1
r1['blockno']=1

In [ ]:
keep_pow=False
keep_phase=True

if keep_pow and keep_phase:
    features = 'all'
elif keep_pow and not keep_phase:
    features = 'pow'
elif not keep_pow and keep_phase:
    features= 'phase'  
features    

In [ ]:
def gen_wide_df(df, f_idx, t_idx, keep_pow, keep_phase):
    """

    :param df:
    :param f_idx:
    :param t_idx:
    :return: df
    """

    df = df.loc[(df['time'].isin(t_idx)) & (df['freqs'].isin(f_idx))]

    # z-score within block
    gb = df.groupby(['elecs', 'freqs', 'blockno'])
    if keep_pow:
        df['zpow'] = gb['pow'].apply(zscore)
    if keep_phase:
        df['sin'] = gb['phase'].apply(np.sin)
        df['cos'] = gb['phase'].apply(np.cos)

    df.drop(['pow', 'phase'], inplace=True, axis=1)

    df = pd.pivot_table(df,
                        index=['class', 'blockno', 'events'],
                        columns=['elecs', 'freqs', 'time'],
                        aggfunc=[lambda x: x])

    elecs = df.columns.get_level_values('elecs')
    freqs = df.columns.get_level_values('freqs')
    time = df.columns.get_level_values('time')
    ncol = len(elecs)
    df.reset_index(drop=False, inplace=True)
    return df, elecs, freqs, time, ncol

In [ ]:
df, elecs, freqs, time, ncol = gen_wide_df(r1, list(range(1,14)), list(range(1,125)), keep_pow, keep_phase)

In [ ]:
view_conds = [f'view{x}' for x in range(1,4)]
cond='view1'

In [ ]:
view_conds.append('response')
view_conds.remove(cond)

In [ ]:
view_conds

In [ ]:
data.columns

In [ ]:
cols_to_drop = [f'view{x}' for x in range(1,4)]

In [ ]:
cols_to_drop.append('response')

In [ ]:
cols_to_drop.remove(cond)

In [ ]:
cols_to_drop

In [ ]:
cols_to_drop = [f'view{x}' for x in range(1,4)]+['response']

In [ ]:
cols_to_drop

In [ ]:
np.array([10,50,100])